In [25]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from neural_network import NeuralNetworkModel, ChessDataset, ChessEvaluationNeuralNetwork

In [26]:
def eval_to_output(eval: str):
    if eval.startswith('#+'):
        return 10000.0
    elif eval.startswith('#-'):
        return -10000.0
    else:
        return float(eval)

In [27]:
df = pd.read_csv("../data/chess_data.csv")

X = df['FEN'].apply(NeuralNetworkModel.fen_to_feature_array).tolist()
X = np.array(X, dtype=np.float32)
y = df['Evaluation'].apply(eval_to_output).astype(np.float32).values.reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


train_dataset = ChessDataset(X_train, y_train)
test_dataset = ChessDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [28]:
model = ChessEvaluationNeuralNetwork(input_size=X.shape[1])
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(20):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader):.4f}")

Epoch 1, Loss: 928230.2840
Epoch 2, Loss: 852193.3510
Epoch 3, Loss: 754411.3974
Epoch 4, Loss: 650673.5570
Epoch 5, Loss: 542772.0093
Epoch 6, Loss: 452902.1570
Epoch 7, Loss: 399005.4623
Epoch 8, Loss: 348588.1237
Epoch 9, Loss: 315708.0359
Epoch 10, Loss: 292502.5063
Epoch 11, Loss: 271078.8285
Epoch 12, Loss: 253628.6395
Epoch 13, Loss: 237043.1342
Epoch 14, Loss: 226820.4332
Epoch 15, Loss: 214957.4691
Epoch 16, Loss: 204605.2868
Epoch 17, Loss: 194569.4345
Epoch 18, Loss: 181961.1724
Epoch 19, Loss: 171081.4029
Epoch 20, Loss: 160785.3674


In [29]:
model.eval()
test_loss = 0.0
with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.item()

avg_test_loss = test_loss / len(test_loader)
rmse_error = np.sqrt(avg_test_loss)
print(f"\nTest RMSE: {rmse_error:.4f}")


Test RMSE: 633.2696


In [30]:
torch.save(model.state_dict(), 'neural_network.pth')